## 1.6 Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

In [100]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "Green Taxi Trip Records", we'll use "For-Hire Vehicle Trip Records".

Download the data for January and February 2021.

Note that you need "For-Hire Vehicle Trip Records", not "High Volume For-Hire Vehicle Trip Records".

Read the data for January. How many records are there?

* 1054112
* 1154112
* 1254112
* 1354112

In [8]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet -O data/fhv_2021-01.parquet
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet -O data/fhv_2021-02.parquet


--2022-05-20 14:04:59--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.89.204
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.89.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11886281 (11M) [binary/octet-stream]
Saving to: ‘data/fhv_2021-01.parquet’

data/fhv_2021-01.pa 100%[===================>]  11.33M  31.5MB/s    in 0.4s    

2022-05-20 14:05:00 (31.5 MB/s) - ‘data/fhv_2021-01.parquet’ saved [11886281/11886281]

--2022-05-20 14:05:00--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.89.204
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.89.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10645466 (10M) [binary/octet-stream]
Saving to: ‘data/fhv_2021-02.parquet’

data/fhv_2021-02.pa

In [84]:
jan_raw = pd.read_parquet('data/fhv_2021-01.parquet')
jan_raw.shape[0]

1154112

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the average trip duration in January?

* 15.16
* 19.16
* 24.16
* 29.16

In [69]:
jan_raw.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [101]:
jan_clean = (jan_raw.assign(duration=lambda df_: (df_['dropOff_datetime'] - df_['pickup_datetime']).dt.total_seconds()/60))
print(jan_clean.duration.mean())

19.1672240937939


## Data preparation

Check the distribution of the duration variable. There are some outliers. 

Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

How many records did you drop? 

## Q3. Missing values

The features we'll use for our model are the pickup and dropoff location IDs. 

But they have a lot of missing values there. Let's replace them with "-1".

What's the fractions of missing values for the pickup location ID? I.e. fraction of "-1"s after you filled the NAs.

* 53%
* 63%
* 73%
* 83

In [103]:
print("PU nan: ", jan_raw.PUlocationID.isna().mean())
print("DO nan: ", jan_raw.DOlocationID.isna().mean())

PU nan:  0.8303067639882438
DO nan:  0.140558282038485


## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix? (The number of columns).

* 2
* 152
* 352
* 525
* 725

In [106]:
train_dicts = df[['DOlocationID', 'PUlocationID']].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train

<1154112x525 sparse matrix of type '<class 'numpy.float64'>'
	with 2308224 stored elements in Compressed Sparse Row format>

In [107]:
X_train.shape

(1154112, 525)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 5.52
* 10.52
* 15.52
* 20.52

In [98]:
def clean_df(raw):
    df = (raw.assign(duration=lambda df_: (df_['dropOff_datetime'] - df_['pickup_datetime']).dt.total_seconds()/60,
                     PUlocationID=lambda df_: df_['PUlocationID'].fillna(-1).astype(str),
                     DOlocationID=lambda df_: df_['DOlocationID'].fillna(-1).astype(str)
                    )
             .query('duration.between(0, 60)')
         )
    return df

    
df = clean_df(jan_raw)

dv = DictVectorizer()
train_dicts = df[['DOlocationID', 'PUlocationID']].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)


target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)



10.62970406167155

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (Feb 2021). 

What's the RMSE on validation?

* 6.01
* 11.01
* 16.01
* 21.01

In [108]:
feb_raw = pd.read_parquet('data/fhv_2021-02.parquet')
df = clean_df(feb_raw)
train_dicts = df[['DOlocationID', 'PUlocationID']].to_dict(orient='records')
X_test = dv.transform(train_dicts)
target = 'duration'
y_test = df[target].values
y_pred = lr.predict(X_test)
mean_squared_error(y_pred, y_test, squared=False)



11.115605145428216

## Submit the results

Submit your results here: https://forms.gle/V8q5rv7QRoZ13Sft6

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline

The deadline for submitting is 23 May 2022, 17:00 CET (Monday). After that, the form will be closed.


